**Building a RAG system using the Mistral-7B model to answer questions based on the Lex Fridman and Andrej Karpathy podcast transcript.**

Load Mistal-7B model with quantized 4-bit

In [1]:
!pip install -q fsspec==2025.3.0 gcsfs datasets transformers trl peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [3]:
!huggingface-cli login # you need token access to mistral model from hugging face


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `mistral judge` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mistral jud

In [36]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_compute_dtype="bfloat16",
                                bnb_4bit_use_double_quant=True,
                                )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map = {"":0},
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Load transcript of the Lex fridman-Andrej Karpathy podcast

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
url = 'https://podscript.ai/podcasts/lex-fridman-podcast/333-andrej-karpathy-tesla-ai-self-driving-optimus-aliens-and-agi'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# View all div classes
for div in soup.find_all('div'):
    print(div.get('class'))

['logo']
['logo-switches']
['breadcrumbs']
['post-meta']
['post-content']
['share-buttons']


In [8]:
transcript_div = soup.find('div', class_='post-content') # 'post-content' is the one containning the actual transcript

transcript_text = transcript_div.get_text(separator='\n') #extract the text

transcript_text

'Listen\nÂ¶\n\n\nTranscript\nÂ¶\nDisclaimer: The transcript that follows has been generated using artificial intelligence. We strive to be as accurate as possible, but minor errors and slightly off timestamps may be present.\nYou can click the timestamp to jump to that time.\nLex Fridman\n\n\n(00:00)\n:\nThe following is a conversation with Andrej Karpathy, previously the director of AI at Tesla, and before that, at OpenAI and Stanford. He is one of the greatest scientist engineers and educators in the history of artificial intelligence.\n(00:19)\n:\nAnd now, a quick few second mention of his sponsor. Check them out in the description. It’s the best way to support this podcast. We got Eight Sleep for naps, BetterHelp for mental health, Fundrise for real estate investing, and Athletic Greens for nutrition. Choose wisely, my friends. And now, onto the full ad reads. As always, no ads in the middle. I tried to make this interesting, but if you skip them, please still check out our sponsor

We clip the unnecessary parts in the intro (sponsors etc.) and the weird text at the bottom (episode info).

In [9]:
start_marker = "And now, dear friends, here’s Andrej Karpathy."
end_marker = "\nEpisode Info"


start_idx = transcript_text.find(start_marker)
end_idx = transcript_text.find(end_marker)


cleaned_transcript =  transcript_text[start_idx + len(start_marker):end_idx]


cleaned_transcript

'\n(05:42)\n:\nWhat is a neural network? And why does it seem to do such a surprisingly good job of learning?\nAndrej Karpathy\n\n\n(05:48)\n:\nWhat is a neural network? It’s a mathematical abstraction of the brain. I would say that’s how it was originally developed. At the end of the day, it’s a mathematical expression, and it’s a fairly simple mathematical expression when you get down to it. It’s basically a sequence of matrix multiplies, whichever link dot products mathematically, and some nonlinearity is thrown in. And so it’s a very simple mathematical expression, and it’s got knobs in it. Many knobs. Many knobs. And these knobs are loosely related to basically the synapses in your brain. They’re trainable, they’re modifiable. And so the idea is we need to find the setting of the knobs that makes the neural net do whatever you want it to do, like classify images and so on.\n(06:27)\n:\nAnd so there’s not too much mystery, I would say, in it. You might think that basically you don’

In [ ]:
# Save to a text file
with open('karpathy_lex_transcript_cleaned.txt', 'w', encoding='utf-8') as f:
    f.write(cleaned_transcript)

print("Transcript saved successfully!")

In [10]:
!pip install -q -U langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [11]:
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [12]:
!pip install -q -U langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

langchain_embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") # you can choose different embedding models from hugging face

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
!pip install -q faiss-cpu # or faiss-gpu if using gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.6 MB/s eta 0:00:00


In [15]:
# Customize text splitting with conversational separators
text_splitter_conv = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=25,
    separators=["\nLex Fridman\n\n\n", "\nAndrej Karpathy\n\n\n"] # this is the format for who is talking, so we devide by it to not mix
)

chunks_conv = text_splitter_conv.split_text(cleaned_transcript)

print(f"{len(chunks_conv)} chunks")

473 chunks


In [16]:
faiss_documents = [Document(page_content=chunk) for chunk in chunks_conv]

In [17]:
faiss_vectorstore = FAISS.from_documents(faiss_documents, langchain_embedder)
faiss_retriever = faiss_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

Without RAG

In [52]:
system_prompt = "Answer the following question appropriately."

query = "What did Andrej karpathy learn from Elon Musk?"

message_template = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]

input = tokenizer.apply_chat_template(message_template)

input = torch.tensor([input]).to(model.device)

output = model.generate(input, max_length=1024)

decoded = tokenizer.decode(output[0])

In [54]:
decoded

"<s> [INST] Answer the following question appropriately.\n\nWhat did Andrej karpathy learn from Elon Musk? [/INST] Andrej Karpathy is a renowned computer scientist and research scientist at Tesla, Inc. He has worked on various projects related to artificial intelligence and deep learning, particularly in the context of autonomous driving. He joined Tesla in 2016, and during his tenure, he has learned several valuable lessons from Elon Musk, the CEO of Tesla. Some of these lessons include:\n\n1. The importance of a long-term vision: Elon Musk is known for his ambitious and long-term vision for Tesla and its various projects. Karpathy has learned the importance of having a clear and long-term vision for research and development projects.\n2. The value of interdisciplinary collaboration: Musk's approach to business involves bringing together people from various disciplines to work on complex problems. Karpathy has learned the importance of interdisciplinary collaboration in research and d

This is good, but not specific to what Andrej said during the conversation. So let's give it some context with the FAISS retriever.

In [26]:
# Now try retrieving with the FAISS retriever
query = "What did Andrej karpathy learn from Elon Musk?"

faiss_results = faiss_retriever.invoke(query)

In [34]:
context = "Context:\n"
for i in range(5):
  context += faiss_results[i].page_content

In [35]:
context

'Context:\nAndrej Karpathy\n\n\n(03:07:13)\n:\nWell, I don’t know about that, actually.Lex Fridman\n\n\n(03:21:56)\n:\nSo aside from the teaching you’re doing now, putting out videos, aside from a potential Godfather part two, with the AGI at Tesla and beyond, what does the future for Andrej Karpathy hold? Have you figured that out yet or no? I mean, as you see through the fog of war, that is all of our future. Do you start seeing silhouettes of what that possible future could look like?Lex Fridman\n\n\n(01:39:10)\n:\nWhat did you learn about machine learning, about engineering, about life, about yourself as one human being from working with Elon Musk?\nAndrej Karpathy\n\n\n(01:39:20)\n:\nI think the most I’ve learned is about how to sort of run organizations efficiently and how to create efficient organizations and how to fight entropy in an organization.Lex Fridman\n\n\n(01:39:30)\n:\nSo human engineering in the fight against entropy.\nAndrej Karpathy\n\n\n(01:39:33)\n:\nYeah, I thin

In [37]:
system_prompt = f"""Your an AI assistant. Your goal is to provide answers about the lex fridman and Andrej Karpathy podcasts, given the provided relevant context with timestamps. Use the context if it is relevant to the question. If you don't know the answer, just say you don't know.
{context}

Please respond to the following question:"""

In [38]:
message_template = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]

input = tokenizer.apply_chat_template(message_template)

In [39]:
input = torch.tensor([input]).to(model.device)

output = model.generate(input, max_length=1024)

decoded = tokenizer.decode(output[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [40]:
decoded

"<s> [INST] Your an AI assistant. Your goal is to provide answers about the lex fridman and Andrej Karpathy podcasts, given the provided relevant context with timestamps. Use the context if it is relevant to the question. If you don't know the answer, just say you don't know.\nContext:\nAndrej Karpathy\n\n\n(03:07:13)\n:\nWell, I don’t know about that, actually.Lex Fridman\n\n\n(03:21:56)\n:\nSo aside from the teaching you’re doing now, putting out videos, aside from a potential Godfather part two, with the AGI at Tesla and beyond, what does the future for Andrej Karpathy hold? Have you figured that out yet or no? I mean, as you see through the fog of war, that is all of our future. Do you start seeing silhouettes of what that possible future could look like?Lex Fridman\n\n\n(01:39:10)\n:\nWhat did you learn about machine learning, about engineering, about life, about yourself as one human being from working with Elon Musk?\nAndrej Karpathy\n\n\n(01:39:20)\n:\nI think the most I’ve lea

In [41]:
start_response = decoded.find("[/INST]")

response = decoded[start_response:]
response

'[/INST] Andrej Karpathy learned about how to run organizations efficiently and create efficient organizations, as well as how to fight entropy in an organization from Elon Musk.</s>'

This is very specific and accurate to what Andrej said during the episode.

We Can also use **llama Index**

In [42]:
!pip -q install llama-index
!pip -q install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.8 MB/s eta 0:00:00


In [43]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import Document
from llama_index.core.postprocessor import SimilarityPostprocessor

In [44]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 512
Settings.chunk_overlap = 25

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [45]:
documents = [Document(text=cleaned_transcript)]

In [46]:
index = VectorStoreIndex.from_documents(documents)

In [67]:
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [68]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [69]:
query = "What did Andrej karpathy learn from Elon Musk?"
response = query_engine.query(query)

In [70]:
context = "Context:\n"
for i in range(top_k):
  context += response.source_nodes[i].text + "\n\n"

In [71]:
context

'Context:\n(01:39:40)\n:\nWhat does the entropy in an organization look like?\nAndrej Karpathy\n\n\n(01:39:42)\n:\nExactly. It’s process, it’s process and inefficiencies in the form of meetings and that kind of stuff. Yeah, meetings, he hates meetings. He keeps telling people to skip meetings if they’re not useful. He basically runs the world’s biggest startups, I would say. Tesla, SpaceX are the world’s biggest startups. Tesla actually has multiple startups. I think it’s better to look at it that way. And so I think he’s extremely good at that. And yeah, he has a very good intuition for streamlining processes, making everything efficient. Best part is no part, simplifying, focusing and just kind of removing barriers, moving very quickly, making big moves. All this is a very startupy sort of seeming things but at scale.\nLex Fridman\n\n\n(01:40:25)\n:\nSo strong drive to simplify. From your perspective, I mean that also probably applies to just designing systems and machine learning an

In [72]:
system_prompt = f"""Your an AI assistant. Your goal is to provide answers about the lex fridman and Andrej Karpathy podcasts, given the provided relevant context with timestamps. Use the context if it is relevant to the question. If you don't know the answer, just say you don't know.
{context}

Please respond to the following question:"""

In [73]:
message_template = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]

input_tokens = tokenizer.apply_chat_template(message_template)

In [76]:
input = torch.tensor([input_tokens]).to(model.device)

output = model.generate(input, max_length=4096)

decoded = tokenizer.decode(output[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [78]:
decoded

"<s> [INST] Your an AI assistant. Your goal is to provide answers about the lex fridman and Andrej Karpathy podcasts, given the provided relevant context with timestamps. Use the context if it is relevant to the question. If you don't know the answer, just say you don't know.\nContext:\n(01:39:40)\n:\nWhat does the entropy in an organization look like?\nAndrej Karpathy\n\n\n(01:39:42)\n:\nExactly. It’s process, it’s process and inefficiencies in the form of meetings and that kind of stuff. Yeah, meetings, he hates meetings. He keeps telling people to skip meetings if they’re not useful. He basically runs the world’s biggest startups, I would say. Tesla, SpaceX are the world’s biggest startups. Tesla actually has multiple startups. I think it’s better to look at it that way. And so I think he’s extremely good at that. And yeah, he has a very good intuition for streamlining processes, making everything efficient. Best part is no part, simplifying, focusing and just kind of removing barri

In [79]:
marker_output = decoded.find("[/INST]")

output = decoded[marker_output:]

In [80]:
output

'[/INST] Andrej Karpathy learned from Elon Musk the importance of having a powerful and intelligent leader who can make big decisions and drive the organization forward. He believes that without such a person, the organization can turn into a collection of committees and become bogged down in decision-making and stakeholder discussions. He also learned the value of simplifying processes and focusing on the fundamental components of solving a problem.</s>'

Again, we get a good answer related to what he said in the podcast.